In [182]:
# Get the Pdf
import PyPDF2
def extract_text_from_pdf(file_path,start_pages,end_page):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(start_pages,end_page):
            text += reader.pages[page].extract_text()
    return text 

In [183]:
# Extract Json from text 
import json  # Import the json module to handle JSON data 
def extract_Json_from_text(text):
    start_idx = text.find("{")
    end_idx = text.rfind("}")
    JsonString = text[start_idx :end_idx+1]
    # Optionally, try to parse the response text as JSON
    try:
        extracted_json = json.loads(JsonString)
        print(JsonString)
    except json.JSONDecodeError:
        print("Failed to decode JSON. Output was:")
        print(JsonString)
    return extracted_json 

In [ ]:
# OpenAI request for interpretation of the regulation
import openai
import os 
openai.api_key = "Dummy" # removed tp push 
def generate_prompt_with_openai(pdf_text):
    PDF_Context = context_text = """
                                You are analyzing a set of regulations designed to guide the placement of smoke detectors and other
                                safety equipment within building floor plans.
                                These regulations are crucial for ensuring safety and compliance with national and local building codes...
                                Additionally, please ensure that any specific terms or phrases mentioned in the PDF document are clearly
                                understood within this context, including any references to sections, diagrams, or specific code provisions.
                                """

    Json_Content = """Please extract and generate a JSON file (without comment) with the following structure from the PDF content provided. 
                    The JSON should contain the following keys and their corresponding values, which represent measurements related to smoke detectors:

                {
                "min_distance_from_wall_or_obstacles_m": null, // Minimum distance from walls or obstacles in meters
                "max_distance_between_alarms_in_corridor_m": null, // Maximum distance between fire alarms in a corridor in meters
                "max_distance_to_corridor_end_m": null, // Maximum distance to the corridor end in meters
                "max_distance_between_alarms_with_partitions_m": null, // Maximum distance between fire alarms in a partitioned corridor in meters
                "max_distance_to_corridor_end_with_partitions_m": null, // Maximum distance to the corridor end in a partitioned corridor in meters
                "max_monitoring_area_per_alarm_m2": null, // Maximum area that can be monitored by a single fire alarm in square meters
                "additional_alarm_area_threshold_m2": null, // Area threshold for requiring an additional fire alarm in square meters
                "ceiling_height_threshold_m": null, // Ceiling height threshold above which multiple levels of alarms are required in meters
                "partition_height_threshold_m": null, // Height of partitions to consider for additional fire alarms in meters
                "ceiling_field_area_threshold_m2": null, // Ceiling field area threshold to determine additional fire alarms in square meters
                "wall_mount_min_distance_below_ceiling_m": null, // Minimum distance below the ceiling for wall-mounted fire alarms in meters
                "wall_mount_max_distance_below_ceiling_m": null, // Maximum distance below the ceiling for wall-mounted fire alarms in meters
                "sloped_ceiling_min_angle_deg": null, // Minimum angle for sloped ceilings in degrees
                "sloped_ceiling_min_distance_from_highest_point_m": null, // Minimum distance from the highest point of sloped ceilings in meters
                "sloped_ceiling_max_distance_from_highest_point_m": null // Maximum distance from the highest point of sloped ceilings in meters
                }

                 Use the information available in the PDF to fill in these values as accurately as possible.""" 
    
    
    Additional_Context = "name other important Information after the Json, that I should consider to integrate in the Json File"

    Content = PDF_Context + Json_Content + pdf_text 
    completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {
            "role": "user",
            "content": Content,
            },
        ],
    )
    print(completion.choices[0].message.content)
    response_text = completion.choices[0].message.content.strip()
    extracted_json = extract_Json_from_text(response_text)  
    # Test the JSON -> print(extracted_json["min_distance_from_wall_or_obstacles_m"]) 

In [185]:
if __name__ == '__main__':
    
    pdf_file_path = r'RulesForSmokeDetectors\\BS5839 PART 1 2017(UKRules).pdf' 
    pdf_text = extract_text_from_pdf(pdf_file_path,5,6)
    print("Generated Output:")
    generate_prompt_with_openai(pdf_text)

Generated Output:
{
    "min_distance_from_wall_or_obstacles_m": 1.5,
    "max_distance_between_alarms_in_corridor_m": null,
    "max_distance_to_corridor_end_m": null,
    "max_distance_between_alarms_with_partitions_m": null,
    "max_distance_to_corridor_end_with_partitions_m": null,
    "max_monitoring_area_per_alarm_m2": null,
    "additional_alarm_area_threshold_m2": null,
    "ceiling_height_threshold_m": null,
    "partition_height_threshold_m": null,
    "ceiling_field_area_threshold_m2": null,
    "wall_mount_min_distance_below_ceiling_m": 25,
    "wall_mount_max_distance_below_ceiling_m": null,
    "sloped_ceiling_min_angle_deg": null,
    "sloped_ceiling_min_distance_from_highest_point_m": null,
    "sloped_ceiling_max_distance_from_highest_point_m": null
}
{
    "min_distance_from_wall_or_obstacles_m": 1.5,
    "max_distance_between_alarms_in_corridor_m": null,
    "max_distance_to_corridor_end_m": null,
    "max_distance_between_alarms_with_partitions_m": null,
    "max_d